In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
# Load your CSV file
import os

# Configure TensorFlow to use GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

df = pd.read_csv(r"C:\Users\abdel\Downloads\data set\SQLiV3.csv")

In [2]:
# Convert 'Label' column to string and then check for non-numeric characters
df['Label'] = df['Label'].astype(str)
df = df[~df['Label'].str.contains('[a-zA-Z]')]
# Convert the 'Label' column to integers
df['Label'] = pd.to_numeric(df['Label'], errors='coerce', downcast='integer')
df['Label'] = df['Label'].astype('float32')
# Drop rows with NaN values in 'Sentence' and 'Label' columns
df = df.dropna(subset=['Sentence', 'Label'])

# Check for NaN values after cleaning
print(df.isna().sum())

Sentence          0
Label             0
Unnamed: 2    30911
Unnamed: 3    30911
dtype: int64


In [3]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize the sentences
max_words = 10000  # Maximum number of words to keep in the vocabulary
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['Sentence'])
train_sequences = tokenizer.texts_to_sequences(train_data['Sentence'])
test_sequences = tokenizer.texts_to_sequences(test_data['Sentence'])

# Pad sequences to have the same length
max_sequence_length = 100  # You can adjust this based on your data and requirements
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Build the improved RNN model
embedding_dim = 32  # Increased dimension of the word embeddings
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Bidirectional(LSTM(64, return_sequences=True)),  # Use Bidirectional LSTM
    Dropout(0.5),  # Add dropout for regularization
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),  # Additional dense layer
    Dropout(0.5),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Binary classification, so using sigmoid activation
])

In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

labels_train = train_data['Label']
labels_test = test_data['Label']

model.fit(train_padded, labels_train, epochs=10, validation_data=(test_padded, labels_test), callbacks=[early_stopping])


# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_padded, labels_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/10
773/773 [==============================] - 65s 61ms/step - loss: 0.0782 - accuracy: 0.9757 - val_loss: 0.0217 - val_accuracy: 0.9951
Epoch 2/10
773/773 [==============================] - 44s 58ms/step - loss: 0.0277 - accuracy: 0.9949 - val_loss: 0.0217 - val_accuracy: 0.9947
Epoch 3/10
773/773 [==============================] - 45s 59ms/step - loss: 0.0182 - accuracy: 0.9963 - val_loss: 0.0405 - val_accuracy: 0.9884
Epoch 4/10
773/773 [==============================] - 41s 54ms/step - loss: 0.0135 - accuracy: 0.9978 - val_loss: 0.0351 - val_accuracy: 0.9908
Epoch 5/10
194/194 [==============================] - 5s 27ms/step - loss: 0.0217 - accuracy: 0.9947
Test Accuracy: 99.47%


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30911 entries, 0 to 31217
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sentence    30911 non-null  object 
 1   Label       30911 non-null  float32
 2   Unnamed: 2  0 non-null      object 
 3   Unnamed: 3  0 non-null      float64
dtypes: float32(1), float64(1), object(2)
memory usage: 1.1+ MB
